In [1]:
import pandas as pd
import numpy as np
import json

In [11]:
db = json.load(open('/Users/KK/Documents/python_for_dataanalysis/pydata-book/datasets/usda_food/database.json'))
db[0].keys()

dict_keys(['nutrients', 'description', 'group', 'manufacturer', 'portions', 'tags', 'id'])

In [12]:
db[0]['description']

'Cheese, caraway'

In [15]:
db[0]['nutrients'][:3]

[{'description': 'Protein',
  'group': 'Composition',
  'units': 'g',
  'value': 25.18},
 {'description': 'Total lipid (fat)',
  'group': 'Composition',
  'units': 'g',
  'value': 29.2},
 {'description': 'Carbohydrate, by difference',
  'group': 'Composition',
  'units': 'g',
  'value': 3.06}]

In [40]:
info_keys = ['description', 'group', 'id', 'manufacturer']
info = pd.DataFrame(db, columns = info_keys)
info.head(5)

,description,group,id,manufacturer
0,"Cheese, caraway",Dairy and Egg Products,1008,
1,"Cheese, cheddar",Dairy and Egg Products,1009,
2,"Cheese, edam",Dairy and Egg Products,1018,
3,"Cheese, feta",Dairy and Egg Products,1019,
4,"Cheese, mozzarella, part skim milk",Dairy and Egg Products,1028,


In [18]:
info.groupby('group').size().head(10)

group
Baby Foods                 209
Baked Products             496
Beef Products              618
Beverages                  278
Breakfast Cereals          403
Cereal Grains and Pasta    183
Dairy and Egg Products     107
Ethnic Foods               165
Fast Foods                 365
Fats and Oils               97
dtype: int64

## flatten nutrients dict and add food id there for future join

In [22]:
len(db[0]['nutrients'])

162

In [28]:
nutrients = pd.DataFrame(db[0]['nutrients'])
nutrients.head(5)

,description,group,units,value
0,Protein,Composition,g,25.18
1,Total lipid (fat),Composition,g,29.20
2,"Carbohydrate, by difference",Composition,g,3.06
3,Ash,Other,g,3.28
4,Energy,Energy,kcal,376.00


In [24]:
nutrients.tail(5)

,description,group,units,value
157,Serine,Amino Acids,g,1.472
158,Cholesterol,Other,mg,93.000
159,"Fatty acids, total saturated",Other,g,18.584
160,"Fatty acids, total monounsaturated",Other,g,8.275
161,"Fatty acids, total polyunsaturated",Other,g,0.830


In [37]:
full_nutrients = pd.DataFrame()
for i in range(len(db)):
    db_temp = pd.DataFrame(db[i]['nutrients'])
    db_temp['id'] = db[i]['id']
    if i == 0:
        full_nutrients = db_temp
    else:
        full_nutrients = pd.concat([full_nutrients, db_temp], ignore_index=True)

In [38]:
full_nutrients.tail(5)

,description,group,units,value,id
389350,"Vitamin B-12, added",Vitamins,mcg,0.000,43546
389351,Cholesterol,Other,mg,0.000,43546
389352,"Fatty acids, total saturated",Other,g,0.072,43546
389353,"Fatty acids, total monounsaturated",Other,g,0.028,43546
389354,"Fatty acids, total polyunsaturated",Other,g,0.041,43546


In [39]:
ndata = pd.merge(full_nutrients, info, on = 'id', how = 'outer')
ndata.head(5)

,description_x,group_x,units,value,id,description_y,group_y,manufacturer
0,Protein,Composition,g,25.18,1008,"Cheese, caraway",Dairy and Egg Products,
1,Total lipid (fat),Composition,g,29.20,1008,"Cheese, caraway",Dairy and Egg Products,
2,"Carbohydrate, by difference",Composition,g,3.06,1008,"Cheese, caraway",Dairy and Egg Products,
3,Ash,Other,g,3.28,1008,"Cheese, caraway",Dairy and Egg Products,
4,Energy,Energy,kcal,376.00,1008,"Cheese, caraway",Dairy and Egg Products,


## which food is most dense in each nutrient

In [42]:
by_nutrient = ndata.groupby(['group_x', 'description_x'])

get_maximum = lambda x: x.loc[x.value.idxmax()]
get_minimum = lambda x: x.loc[x.value.idxmin()]

max_food = by_nutrient.apply(get_maximum)[['value', 'description_y']]
max_food

value  \
group_x     description_x                                
Amino Acids Alanine                              8.009   
            Arginine                             7.436   
            Aspartic acid                       10.203   
            Cystine                              1.307   
            Glutamic acid                       17.452   
            Glycine                             19.049   
            Histidine                            2.999   
            Hydroxyproline                       0.803   
            Isoleucine                           4.300   
            Leucine                              7.200   
            Lysine                               6.690   
            Methionine                           1.859   
            Phenylalanine                        4.600   
            Proline                             12.295   
            Serine                               4.600   
            Threonine                            3.300   
            Tryptophan                           1.600   
            Tyrosine                             3.300   
            Valine                               4.500   
Composition Adjusted Protein                    12.900   
            Carbohydrate, by difference        100.000   
            Fiber, total dietary                79.000   
            Protein                             88.320   
            Sugars, total                       99.800   
            Total lipid (fat)                  100.000   
            Water                              100.000   
Elements    Calcium, Ca                       7364.000   
            Copper, Cu                          15.050   
            Fluoride, F                        584.000   
            Iron, Fe                            87.470   
...                                                ...   
Vitamins    Cryptoxanthin, beta               6186.000   
            Dihydrophylloquinone               103.800   
            Folate, DFE                       2630.000   
            Folate, food                      2340.000   
            Folate, total                     2340.000   
            Folic acid                        1538.000   
            Lutein + zeaxanthin              39550.000   
            Lycopene                         46260.000   
            Menaquinone-4                       33.200   
            Niacin                              97.000   
            Pantothenic acid                    35.000   
            Retinol                          30000.000   
            Riboflavin                          14.300   
            Thiamin                             20.000   
            Tocopherol, beta                     6.490   
            Tocopherol, delta                   30.880   
            Tocopherol, gamma                  100.880   
            Vitamin A, IU                   100000.000   
            Vitamin A, RAE                   30000.000   
            Vitamin B-12                        98.890   
            Vitamin B-12, added                 24.000   
            Vitamin B-6                         12.000   
            Vitamin C, total ascorbic acid    2400.000   
            Vitamin D                        10000.000   
            Vitamin D (D2 + D3)                250.000   
            Vitamin D2 (ergocalciferol)         28.100   
            Vitamin D3 (cholecalciferol)        27.400   
            Vitamin E (alpha-tocopherol)       149.400   
            Vitamin E, added                    46.550   
            Vitamin K (phylloquinone)         1714.500   

                                                                                description_y  
group_x     description_x                                                                      
Amino Acids Alanine                                         Gelatins, dry powder, unsweetened  
            Arginine                                             Seeds, sesame flour, low-fat  
          